In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
import os
import spacy
from tqdm import tqdm
import xml.etree.ElementTree as et

In [3]:
import copy

def calc_precision(tp, fp):
    return tp/(tp + fp)

def calc_recall(tp, fn):
    return tp/(tp + fn)

def calc_fscore(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

def evaluate(gold_truth_labels, predictions):
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    for gold, pred in zip(gold_truth_labels, predictions):
        
        tp_tmp, fp_tmp, fn_tmp, fns_temp, fps_temp  = evaluate_one_article(gold, pred)
        
        tp += tp_tmp
        fp += fp_tmp
        fn += fn_tmp
        
        fns.extend(fns_temp)
        fps.extend(fps_temp) 
        
    precision = calc_precision(tp, fp)
    recall = calc_recall(tp, fn)
    f_score = calc_fscore(precision, recall)    
    
    print(f'fp: {fp} | tp: {tp} | fn: {fn}')
    print(f'precision: {precision:.3f} | recall: {recall:.3f} | f-score: {f_score:.3f}')
    
    return fps, fns  
    

def evaluate_one_article(gold_truth, prediction):
    
    gold = gold_truth['entities'].copy()
    pred = prediction['entities'].copy()
    
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    
    i = 0
    
    while len(gold) > 0 and len(pred) > 0:
        i += 1

        # Check if the first two elements are the same
        if gold[0] == pred[0]:
            tp += 1
            gold.pop(0)
            pred.pop(0)
        
        else:
            # Grab the first appearing element
            element, source = (gold[0], 'gold') if gold[0]['start_pos'] < pred[0]['start_pos'] else (pred[0], 'pred')
            
            # Remove the element first appearing element
            if source == 'gold':
                fn += 1
                fns.append(element['text'])
                gold.remove(element)
            elif source == 'pred':
                fp += 1
                fps.append(element['text'])
                pred.remove(element)
    
    if len(gold) > 0:
        fn += 1
    elif len(pred) > 0:
        fp += 1
        
    return tp, fp, fn, fns, fps       

def run_flair(text):

    # make a sentence
    sentence = Sentence(text)

    # run NER over sentence
    tagger.predict(sentence)
    
    for entity in sentence.to_dict(tag_type='ner')['entities']:
        print(entity)

In [4]:
def load_file(file_path):
    """
    Loads file and returns all the articles
    """
    # Load the data
    tree = et.parse(file_path)
    root = tree.getroot()

    return root

def process_article(article, filtered, file_path):
    """
    Takes article and process into desired structure
    """
    if 'GeoWebNews' in file_path:
        if filtered:
            return {'text': article.find('text').text, 
                    'entities': sorted([{'text': top.find('extractedName').text, 
                                                      'start_pos': int(top.find('start').text), 
                                                      'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym') 
                                                     if top.find('latitude') != None and top.find('longitude') != None], key=lambda k: k['start_pos'])}
        
        else:
            return {'text': article.find('text').text, 
                    'entities': sorted([{'text': top.find('extractedName').text, 
                                                      'start_pos': int(top.find('start').text), 
                                                      'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')], key=lambda k: k['start_pos'])}
    
    
    elif not filtered:
        return {'text': article.find('text').text,
                'entities': sorted([{'text': top.find('phrase').text,
                            'start_pos': int(top.find('start').text),
                            'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
                                 ], key=lambda k: k['start_pos'])}
        
    else:
        return {'text': article.find('text').text,
                'entities': sorted([{'text': top.find('phrase').text,
                            'start_pos': int(top.find('start').text),
                            'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
                             if top.find('gaztag/lat') != None and top.find('gaztag/lon') != None
                                 ], key=lambda k: k['start_pos'])}

def process_articles(root, filtered, file_path):
    """
    Takes articles and processes them into desired structure
    """
    data = []
    
    for article in root:
        
        data.append(process_article(article, filtered, file_path))
    
    return data

def prepare_data(file_path, filtered):
    
    root = load_file(file_path)
    
    data = process_articles(root, filtered, file_path)
    
    return data

In [5]:
def make_predictions_spacy(data):
    
    predictions = []
    
    texts = [article['text'] for article in data]
    
    for doc in tqdm(nlp.pipe(texts)):
        # Do something with the doc here
        
        pred = {'entities': [{'text': ent.text, 
                              'start_pos': len(doc[0:ent.end].text) - len(doc[ent.start]), 
                              'end_pos': len(doc[0:ent.end].text)} for ent in doc.ents if 
                                                             ent.label_ == 'LOC' or ent.label_ == 'GPE']}
        
        if pred:
            pred['text'] = doc
            
            predictions.append(pred)
        
    return predictions


### Load the default transfomer spacy model

In [6]:
# Only enable the ner tagger
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

## TR-News

In [7]:
# Get file path LGL dataset
file_path = '../../data/TR-News/TR-News.xml'

In [8]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for TR-News

In [9]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

118it [02:26,  1.24s/it]
118it [02:26,  1.24s/it]


#### Results TR-News & Comparison

In [10]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 236 | tp: 748 | fn: 498
precision: 0.760 | recall: 0.600 | f-score: 0.671


In [11]:
fps

['U.S.-led',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'New York',
 'New York',
 'the United States',
 'the United States',
 'the United States',
 'College Station',
 'West Chester',
 'the United States',
 'Southern District of Ohio',
 'South Carolina',
 'South Carolina',
 'New Milford',
 'Downtown Eastside',
 "San Francisco's",
 'Rhineland-Palatinate',
 'the South Pacific',
 'Singapore, Singapore',
 'the Central Coast',
 'West Coast',
 'Etobicoke',
 'Man',
 'Churchill',
 'Churchill',
 'Churchill',
 'Winnipeg',
 'Little Portugal',
 'Phoenix',
 'Phoenix',
 'Gatineau',
 'Que',
 'Ottawa',
 'the United States',
 'New York',
 'Giza',
 'Ont',
 'the Greater Toronto Area',
 'British Columbia',
 'Ottawa',
 'Ottawa',
 'Mississauga',
 'Ontario',
 'Ontario',
 'Toronto',
 'GTA',
 'Toronto',
 'Ontario',
 'Mississauga',
 'Costa Rica',
 'Costa Rica',
 'Costa Rica',
 'Costa Rica

In [12]:
fns

['Turkish',
 'Turkish',
 'Syrian',
 'Syrian',
 'U.S.',
 'Turkish',
 'Turkish',
 'Russian',
 'Syrian',
 'Russian',
 'Turkey',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Russian',
 'Turkish',
 'Russian',
 'Russian',
 'Pyongyang',
 'Russian',
 'Syrian',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'Granville County',
 'New York',
 'New York',
 'New York City',
 'United States',
 'United States',
 'United States',
 'White House',
 'Texas',
 'College Station',
 'Texas',
 'Texas',
 'U.S.',
 'West Chester',
 'Xavier University',
 'United States',
 'British',
 'European',
 'Ohio',
 'New York',
 'U.S.',
 'United States',
 'Wisconsin',
 'New York',
 'Michigan',
 'Pennsylvania',
 'South Carolina',
 'South Carolina',
 'Cuban',
 'Bantam',
 'New Milford',
 'New Milford',
 'Red Deer',
 'Russian',
 'Russian',

In [13]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 236 | tp: 748 | fn: 530
precision: 0.760 | recall: 0.585 | f-score: 0.661


In [14]:
fps

['U.S.-led',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'New York',
 'New York',
 'the United States',
 'the United States',
 'the United States',
 'College Station',
 'West Chester',
 'the United States',
 'Southern District of Ohio',
 'South Carolina',
 'South Carolina',
 'New Milford',
 'Downtown Eastside',
 "San Francisco's",
 'Rhineland-Palatinate',
 'the South Pacific',
 'Singapore, Singapore',
 'the Central Coast',
 'West Coast',
 'Etobicoke',
 'Man',
 'Churchill',
 'Churchill',
 'Churchill',
 'Winnipeg',
 'Little Portugal',
 'Phoenix',
 'Phoenix',
 'Gatineau',
 'Que',
 'Ottawa',
 'the United States',
 'New York',
 'Giza',
 'Ont',
 'the Greater Toronto Area',
 'British Columbia',
 'Ottawa',
 'Ottawa',
 'Mississauga',
 'Ontario',
 'Ontario',
 'Toronto',
 'GTA',
 'Toronto',
 'Ontario',
 'Mississauga',
 'Costa Rica',
 'Costa Rica',
 'Costa Rica',
 'Costa Rica

In [15]:
fns

['Turkish',
 'Turkish',
 'Syrian',
 'Syrian',
 'U.S.',
 'Turkish',
 'Kurdish',
 'Turkish',
 'Russian',
 'Syrian',
 'Russian',
 'Turkey',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Russian',
 'Turkish',
 'Russian',
 'Russian',
 'Pyongyang',
 'Russian',
 'Syrian',
 'North Carolina',
 'West Virginia',
 'Granville County',
 'West Virginia',
 'West Virginia',
 'North Carolina',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'West Virginia',
 'Granville County',
 'New York',
 'New York',
 'New York City',
 'United States',
 'United States',
 'United States',
 'White House',
 'Texas',
 'College Station',
 'Texas',
 'Texas',
 'U.S.',
 'West Chester',
 'Xavier University',
 'United States',
 'British',
 'European',
 'Ohio',
 'New York',
 'U.S.',
 'United States',
 'Wisconsin',
 'New York',
 'Michigan',
 'Pennsylvania',
 'South Carolina',
 'South Carolina',
 'Cuban',
 'Bantam',
 'New Milford',
 'New Milford',
 'Red Deer',
 'Russian',

## LGL

In [16]:
# Get file path LGL dataset
file_path = '../../data/LGL/LGL.xml'

In [17]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for LGL

In [18]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

588it [08:53,  1.10it/s]
588it [08:53,  1.10it/s]


#### Results LGL & Comparison

In [19]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 1144 | tp: 2324 | fn: 2002
precision: 0.670 | recall: 0.537 | f-score: 0.596


In [20]:
fps

['Kelleyland',
 'Pointe Coupee',
 'Pelican Rapids',
 'Grant',
 'Benson',
 'Otter Tail',
 'Parkers Prairie',
 'Parkers Prairie',
 'the Red River',
 'Fargo-Moorhead',
 'Red River',
 'the Red River Valley',
 'North Dakota',
 'Oakport Township',
 'the Red River',
 'Red River',
 'Lake Vermont',
 'Nokomis',
 'New Haven',
 'Sioux Falls',
 'New Haven',
 'Old Town',
 'Old Town',
 'New York City',
 'Old Town',
 'Northwest D.C.',
 'Southern Sudan',
 'Sub-Saharan Africa',
 'Godapitiya',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Cairo',
 'Port Said',
 'Misri',
 'the Middle East',
 'Gulf',
 'Gulf',
 'US',
 'US',
 'US',
 'the West Bank',
 'Gulf',
 'US',
 'Gulf',
 'the Middle East',
 'US',
 'Sharm El-Sheikh',
 'Rolling Meadows',
 'Rolling Meadows',
 'North Monticello',
 'DuPage County',
 'Lombard',
 'Babcock Grove',
 'Glen Ellyn',
 'Macon County',
 'Hudson',
 'Evergreen Lake',
 'Decatur',
 'McLean County',
 'Des Plaines',
 'North Texas',
 'Ta

In [21]:
fns

['Cottonport',
 'Pointe Coupee',
 'Mansfield',
 'Shreveport',
 'Cook',
 'Minneapolis',
 'Minnesota',
 'Minnesota',
 'Chisholm',
 'Elbow Lake',
 'Twin Lakes',
 'Otter Tail County',
 'Pelican Rapids',
 'Star Lake',
 'Douglas County',
 'Kent',
 'Minnesota',
 'Minnesota',
 'Otter Tail',
 'Parkers Prairie',
 'Parkers Prairie',
 'Parkers Prairie',
 'Alexandria',
 'Red River',
 'Fargo',
 'Moorhead',
 'Red River',
 'North Dakota',
 'Oakport Township',
 'Red River',
 'Minnesota',
 'Fargo',
 'Red River',
 'Fargo',
 'Douglas County',
 'Lake Vermont',
 'Freeborn Lake',
 'Elbow Lake',
 'Alexandria',
 'New Haven',
 'Sioux Falls',
 'New Haven',
 'Sioux Falls',
 'Alexandria',
 'Alexandria',
 'New York City',
 'D.C.',
 'D.C.',
 'Sudanese',
 'Sudan',
 'Chinese',
 'Africa',
 'African',
 'Darfur',
 'Egyptian',
 'Egyptian',
 'Sudan',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Egyptian',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Egyptian',
 'Sudanese',
 'Suda

In [22]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 1091 | tp: 2391 | fn: 2475
precision: 0.687 | recall: 0.491 | f-score: 0.573


In [23]:
fps

['Kelleyland',
 'Pointe Coupee',
 'Pelican Rapids',
 'Grant',
 'Otter Tail',
 'Parkers Prairie',
 'Parkers Prairie',
 'the Red River',
 'Fargo-Moorhead',
 'Red River',
 'the Red River Valley',
 'North Dakota',
 'Oakport Township',
 'the Red River',
 'Red River',
 'Lake Vermont',
 'Freeborn Lake',
 'New Haven',
 'Sioux Falls',
 'New Haven',
 'Old Town',
 'Old Town',
 'New York City',
 'Old Town',
 'Northwest D.C.',
 'Southern Sudan',
 'Sub-Saharan Africa',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 'Sri Lanka',
 "Sri Lanka's",
 'Cairo',
 'Port Said',
 'Misri',
 'the Middle East',
 'Gulf',
 'Gulf',
 'US',
 'US',
 'US',
 'the West Bank',
 'Gulf',
 'US',
 'Gulf',
 'the Middle East',
 'US',
 'Sharm El-Sheikh',
 'Rolling Meadows',
 'Rolling Meadows',
 'North Monticello',
 'DuPage County',
 'Lombard',
 'Babcock Grove',
 'Glen Ellyn',
 'Macon County',
 'Hudson',
 'Evergreen Lake',
 'Decatur',
 'McLean County',
 'Des Plaines',
 'North Texas',
 'Tarran

In [24]:
fns

['Cottonport',
 'Memphis St.',
 'Augusta St.',
 'Pointe Coupee',
 'Mansfield',
 'Shreveport',
 'DeSoto Parish',
 'Cook',
 'Minneapolis',
 'Minnesota',
 'Minnesota',
 'Chisholm',
 'Elbow Lake',
 'Highway 200',
 'Mahnomen County Road',
 'Mahnomen County Road',
 'Twin Lakes',
 'Highway 10',
 'Otter Tail County',
 'Highway 108',
 'Pelican Rapids',
 'Star Lake',
 'Otter Tail/Grant',
 'Grant/Wilkin',
 'County Road 43',
 'Highway 114',
 'Douglas County',
 'Lake Mary',
 'Kent',
 'Minnesota',
 'Minnesota',
 'Otter Tail',
 'Parkers Prairie',
 'Parkers Prairie',
 'Parkers Prairie',
 'Alexandria',
 'Red River',
 'Fargo',
 'Moorhead',
 'Red River',
 'Red River Valley',
 '40th Avenue South',
 'North Dakota',
 'Oakport Township',
 'Red River',
 'Minnesota',
 'Fargo',
 'Red River',
 'Fargo',
 'Douglas County',
 'County Road 35',
 'Lake Vermont',
 'County Road 56',
 'County Road 15',
 'County Road 96',
 'Freeborn Lake',
 'North Nokomis Street',
 'Darling Avenue',
 'County Road 109',
 'Elbow Lake',
 'Va

## GeoWebNews

In [25]:
# Get file path LGL dataset
file_path = '../../data/GeoWebNews/GeoWebNews.xml'

In [26]:
data_all_toponyms = prepare_data(file_path, filtered=False)
data_filtered_toponyms = prepare_data(file_path, filtered=True)

#### Predictions for GeoWebNews

In [27]:
predictions_all_toponyms = make_predictions_spacy(data_all_toponyms)
predictions_filtered_toponyms = make_predictions_spacy(data_filtered_toponyms)

200it [04:05,  1.23s/it]
200it [04:07,  1.24s/it]


#### Results GeoWebNews & Comparison

In [28]:
# only toponyms w/ lat/long
fps, fns = evaluate(data_filtered_toponyms, predictions_filtered_toponyms)

fp: 385 | tp: 1236 | fn: 1253
precision: 0.762 | recall: 0.497 | f-score: 0.601


In [29]:
fps

['French Quarter',
 'the Mississippi River',
 'the Faubourg Marigny',
 'Elysian Fields',
 'Elysian Fields',
 'Smoky Mary',
 'Granville',
 'Somerville',
 'Lake Manassas',
 'Vienna',
 'North Carolina',
 'Bollywood',
 'Fire Island',
 'Fort Salonga',
 'New Orleans',
 'New York',
 'South Korea',
 'New York City',
 'New Orleans',
 'the Commonwealth of Independent States (',
 "the Islamic Republic's",
 'CIS',
 'West Coast',
 'North America',
 'Treasure Beach',
 'Sanshaawes',
 'Epiphaneia',
 'Mubtaahij',
 'Morris County',
 'Morris County',
 'New York',
 'New York',
 'NEW JERSEY',
 'Washington',
 'D.C.',
 'the Middle East',
 'Sheikh Akil',
 'the United States',
 'West',
 'the United Kingdom',
 'the United States',
 'VNA',
 'Asian-Pacific region',
 'Asia-Pacific',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'the United States',
 'the United States',
 'Washington Township',
 'South Africa',
 'The United States',
 'United

In [30]:
fns

['Louisiana',
 'French',
 'French Quarter',
 'Mississippi River',
 'Faubourg Marigny',
 'German',
 'Irish',
 'Rue d’Enghein',
 'Lafayette',
 'Almonaster',
 'Franklin',
 'Marigny Plantation',
 'Press Street',
 'Franklin',
 'Faubourg D’Aunoy',
 'Chartres',
 'Franklin',
 'Methodist church',
 'Royal Street',
 'Royal Street',
 'Rue Casa Calvo',
 'Faubourg Marigny',
 '2231 Royal',
 'Greek',
 'Elysian Fields',
 'Royal Street',
 'Dauphine',
 'Desire Street',
 'Elysian Fields',
 'New Orleans Railways and Light Company Claiborne Power House',
 'French',
 'Marigny Canal',
 'Washington Square',
 'Champs-Élysées',
 'Elysian Fields',
 'Pontchartrain Railroad',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Syrian',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Lake Manassas',
 'Robert Trent Jones Golf Club',
 'Stonewall Golf Club',
 'Stonewall',
 'Virginia Gateway',
 'Haymarket Village Center',
 'Buckland Elementary',
 'Ronald Wilson Reagan Middle',

In [31]:
# all toponyms
fps, fns = evaluate(data_all_toponyms, predictions_all_toponyms)

fp: 379 | tp: 1243 | fn: 4365
precision: 0.766 | recall: 0.222 | f-score: 0.344


In [32]:
fps

['French Quarter',
 'the Mississippi River',
 'the Faubourg Marigny',
 'Elysian Fields',
 'Elysian Fields',
 'Smoky Mary',
 'Lake Manassas',
 'Vienna',
 'North Carolina',
 'Bollywood',
 'Fire Island',
 'Fort Salonga',
 'New Orleans',
 'New York',
 'South Korea',
 'New York City',
 'New Orleans',
 'the Commonwealth of Independent States (',
 "the Islamic Republic's",
 'CIS',
 'West Coast',
 'North America',
 'Treasure Beach',
 'Sanshaawes',
 'Epiphaneia',
 'Mubtaahij',
 'Morris County',
 'Morris County',
 'New York',
 'New York',
 'NEW JERSEY',
 'Washington',
 'D.C.',
 'US',
 'the Middle East',
 'Sheikh Akil',
 'the United States',
 'the United Kingdom',
 'the United States',
 'VNA',
 'Asian-Pacific region',
 'Asia-Pacific',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'Tamil Nadu',
 'the United States',
 'the United States',
 'Washington Township',
 'South Africa',
 'South Venezuela',
 'The United States',
 'United States',
 

In [33]:
fns

['area',
 'plantation',
 'mansion',
 'substation',
 'Louisiana',
 'Louisiana Purchase',
 'parcel',
 'French',
 'plat',
 'French Quarter',
 'Mississippi River',
 'squares',
 'neighborhood',
 'city',
 'Faubourg Marigny',
 'community',
 'African Americans',
 'German',
 'Irish',
 'populations',
 'blocks',
 'residents',
 'blocks',
 'intersection',
 'mills',
 'plant',
 'stables',
 'factory',
 'barn',
 'streets',
 'Rue d’Enghein',
 'street',
 'Lafayette',
 'Almonaster',
 'Franklin',
 'avenue',
 'Marigny Plantation',
 'faubourg',
 'Press Street',
 'area',
 'Franklin',
 'Faubourg D’Aunoy',
 'neighbor',
 'building',
 'Chartres',
 'Franklin',
 'Methodist church',
 'restaurant',
 'street',
 'complex',
 'Royal Street',
 'Royal Street',
 'Rue Casa Calvo',
 'Faubourg Marigny',
 '2231 Royal',
 '2231 Royal',
 'townhouse',
 'basement',
 'Greek',
 'structures',
 'Elysian Fields',
 'Royal Street',
 'Dauphine',
 'Desire Street',
 'system',
 'neighborhoods',
 'edifice',
 'Elysian Fields',
 'New Orleans Rail